### init

In [55]:
import turicreate
## load dataset SFrame
people = turicreate.SFrame('../Module1/people_wiki.sframe')
## count the words and store in new row
people['word_count'] = turicreate.text_analytics.count_words(people['text'])
## compute TF/ID amd store in the new row
people['tfidf'] = turicreate.text_analytics.tf_idf(people['text'])
##build NN model using tf/id as parameter
knn_model = turicreate.nearest_neighbors.create(people,features=['tfidf'],label='name')


Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [24]:
#create KNN models
knn_model_tfidf = turicreate.nearest_neighbors.create(people,features=['tfidf'],label='name',distance='cosine')
knn_model_word = turicreate.nearest_neighbors.create(people,features=['word_count'],label='name',distance='cosine')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

### 1. Palavras com maior número de palavras para Elton John

In [56]:
elton_john =  people[people['name'] == 'Elton John']
## word count
elton_john_words = elton_john[['word_count']].stack('word_count', new_column_name = ['word','count']).sort('count',ascending=False)
print(elton_john_words[0:3])

+------+-------+
| word | count |
+------+-------+
| the  |  27.0 |
|  in  |  18.0 |
| and  |  15.0 |
+------+-------+
[3 rows x 2 columns]



### 2. Principais palavras TF-IDF para Elton John

In [57]:
## tf/id count
elton_john_tfidf = elton_john[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)
print(elton_john_tfidf[0:3])

+-----------+--------------------+
|    word   |       tfidf        |
+-----------+--------------------+
|  furnish  | 18.38947183999428  |
|   elton   | 17.482320270031995 |
| billboard | 17.30368095754203  |
+-----------+--------------------+
[3 rows x 2 columns]



### 3. A distância cosseno entre os artigos de "Elton John" e "Victoria Beckham" (representada com TF-IDF) está dentro de qual intervalo?

In [30]:
beckham = people[people['name'] == 'Victoria Beckham']
# 1st comparation
comp_beckham = turicreate.distances.cosine(elton_john['tfidf'][0],beckham['tfidf'][0])
print(comp_beckham)

0.9567006376655429


### 4. A distância de cosseno entre os artigos de 'Elton John' e 'Paul McCartney' (representados com TF-IDF) está dentro de qual intervalo?

In [31]:
paul = people[people['name'] == 'Paul McCartney']
comp_paul = turicreate.distances.cosine(elton_john['tfidf'][0],paul['tfidf'][0])
print(comp_paul)

0.8250310029221779


### 5. Quem é mais próximo de "Elton John", "Victoria Beckham" ou "Paul McCartney"?

In [54]:
if comp_beckham < comp_paul:
    print(beckham['name'])
else:
    print(paul['name'])

['Paul McCartney', ... ]


### 6. Quem é o vizinho de distância cosseno mais próximo de "Elton John" usando a contagem de palavras brutas?

In [49]:
knn_model_word.query(elton_john)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 6.563ms      |

| Done         |         | 100         | 219.48ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.220446049250313e-16,1
0,Cliff Richard,0.16142415258967036,2
0,Sandro Petrone,0.16822542751041114,3
0,Rod Stewart,0.16832716558706107,4
0,Malachi O'Doherty,0.177315545978884,5


### 7. Quem é o vizinho de distância cosseno mais próximo de "Elton John" usando TF-IDF?

In [50]:
knn_model_tfidf.query(elton_john)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 7.901ms      |

| Done         |         | 100         | 355.596ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.220446049250313e-16,1
0,Rod Stewart,0.7172196678927374,2
0,George Michael,0.7476009989692848,3
0,Sting (musician),0.7476719544306141,4
0,Phil Collins,0.7511932487904706,5


### 8. Quem é o vizinho de distância cosseno mais próximo de "Victoria Beckham" usando a contagem de palavras brutas?

In [52]:
knn_model_word.query(beckham)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 11.049ms     |

| Done         |         | 100         | 268.328ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.220446049250313e-16,1
0,Mary Fitzgerald (artist),0.20730703611504997,2
0,Adrienne Corri,0.21450978278754795,3
0,Beverly Jane Fry,0.21746646874079278,4
0,Raman Mundair,0.21769547499150488,5


### 8. Quem é o vizinho de distância cosseno mais próximo de "Victoria Beckham" usando TF-IDF?

In [53]:
knn_model_tfidf.query(beckham)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 9.177ms      |

| Done         |         | 100         | 563.26ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.1102230246251565e-16,1
0,David Beckham,0.5481696102632145,2
0,Stephen Dow Beckham,0.7849867068283364,3
0,Mel B,0.8095855234085036,4
0,Caroline Rush,0.81982642291868,5
